In [1]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 2.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, datasets

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# Hyperparameters
batch_size = 256
patch_size = 4  # 4x4 patches
num_patches = (32 // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [4]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

170498071/170498071 [==============================] - 4s 0us/step


In [5]:
# Patch creation utility
def create_patches(x, patch_size):
    batch_size = tf.shape(x)[0]
    patches = tf.image.extract_patches(
        images=x,
        sizes=[1, patch_size, patch_size, 1],
        strides=[1, patch_size, patch_size, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, -1, patch_dims])
    return patches

In [6]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim)

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [7]:
# Build the model
def create_vit_classifier():
    inputs = layers.Input(shape=(32, 32, 3))
    patches = create_patches(inputs, patch_size)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = layers.Dense(units=transformer_units[0], activation=tf.nn.gelu)(x3)
        x3 = layers.Dense(units=transformer_units[1], activation=tf.nn.gelu)(x3)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    features = layers.Dense(units=mlp_head_units[0], activation=tf.nn.gelu)(representation)
    features = layers.Dropout(0.5)(features)
    features = layers.Dense(units=mlp_head_units[1], activation=tf.nn.gelu)(features)
    outputs = layers.Dense(units=10)(features)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
# Compile and train the model
model = create_vit_classifier()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

In [9]:
history=model.fit(x_train, y_train, batch_size=batch_size, epochs=60, validation_split=0.1)

Epoch 1/60
176/176 [==============================] - 53s 143ms/step - loss: 2.5189 - accuracy: 0.1529 - val_loss: 2.0291 - val_accuracy: 0.2238
Epoch 2/60
176/176 [==============================] - 24s 138ms/step - loss: 1.8864 - accuracy: 0.2884 - val_loss: 1.6779 - val_accuracy: 0.3802
Epoch 3/60
176/176 [==============================] - 25s 140ms/step - loss: 1.6634 - accuracy: 0.3838 - val_loss: 1.5747 - val_accuracy: 0.4192
Epoch 4/60
176/176 [==============================] - 24s 137ms/step - loss: 1.4818 - accuracy: 0.4575 - val_loss: 1.3490 - val_accuracy: 0.5130
Epoch 5/60
176/176 [==============================] - 24s 137ms/step - loss: 1.3607 - accuracy: 0.5062 - val_loss: 1.3235 - val_accuracy: 0.5300
Epoch 6/60
176/176 [==============================] - 24s 137ms/step - loss: 1.2893 - accuracy: 0.5325 - val_loss: 1.2150 - val_accuracy: 0.5774
Epoch 7/60
176/176 [==============================] - 24s 139ms/step - loss: 1.2079 - accuracy: 0.5650 - val_loss: 1.1673 - val_ac

In [10]:
def plot_history(history, filename, model_name):
  best_epoch = history.history['val_loss'].index(min(history.history['val_loss']))
  fig, ax1 = plt.subplots(figsize=(18,8))
  plt.title(' '.join([model_name, 'Maximum Accuracy is %1.4f' % max(history.history['val_accuracy'])]))
  ax1.set_xlabel('Epochs')
  ax1.set_xticks(range(1,61))
  ax1.set_ylabel('Loss')
  ax1.plot(range(1,61), history.history['loss'], 'r', label='Train Loss')
  ax1.plot(range(1,61), history.history['val_loss'], 'orange', label='Test Loss')
  ax1.axvline(best_epoch, color='m', lw=4, alpha=0.5, label='Best epoch')
  ax1.legend()

  ax2 = ax1.twinx()
  ax2.set_ylabel('Accuracy')
  ax2.plot(range(1,61), history.history['accuracy'], 'g', label='Train Accuracy')
  ax2.plot(range(1,61), history.history['val_accuracy'], 'b', label='Test Accuracy')
  ax2.legend()

  plt.savefig(filename)
  plt.show()

#plotting learning curves and labelling them
filename='Model.png'
model_name = 'Model'
plot_history(history, filename, model_name)

print("Maximum Test Accuracy: ", max(history.history['val_accuracy']))
print("Maximum Train Accuracy: ", max(history.history['accuracy']))